In [2]:
# Install uncommon packages (will supply requements.txt later)
#!pip install git+https://github.com/boudinfl/pke.git
#!pip install wordninja
#!pip install pyenchant

# For macos:
#brew install cmake
#git clone git@github.com:lloyd/yajl.git
#cd yajl
#./configure && make install


import pandas as pd
import numpy as np
import os
import json
import wordninja
from os import listdir
import re
import sys
import ast
import pke
import enchant
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import requests
import pickle
import datetime
import ijson.backends.yajl2_cffi as ijson


#import pycorpora
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
d = enchant.Dict("en_US") # ENGLISH Dictionary


# CHANGE THE PATH TO A DATASET HERE
basedir = os.path.join(os.sep, "media", "tie-server", "DATA", "Jens", "Crunchbase")
path_to_embedrank_repo = os.path.join("ai-research-keyphrase-extraction") # relative (without "../") or absolute path


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tie-server/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/tie-server/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tie-
[nltk_data]     server/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organization_descriptions.csv"), index_col="uuid")
#data.iloc[:,1].to_csv('CB_Export_17_08_07/descr.txt')
print("loaded descriptions")
data1 = pd.read_csv(os.path.join(basedir, "CB_export", "cb_organizations.csv"), low_memory=False, index_col="uuid")
print("loaded org infos")

loaded descriptions
loaded org infos


In [3]:
# Merge 2 csv files
# we can time to see which is more effective
#result = pd.concat([data, data1], axis=1, sort=False)
result1 = pd.merge(data, data1, how='inner', left_index=True, right_index=True)
print("merged")
# Delete all companies that do not have any descriptions
result1 = result1.dropna(subset=["description"])

merged


In [4]:
#print(((result.shape[0] - result1.shape[0])/result.shape[0]*100), "% of companies do not have any description")

In [7]:
result1.head()

,description,company_name,primary_role,permalink,domain,homepage_url,country_code,state_code,region,city,...,phone,facebook_url,linkedin_url,cb_url,logo_url,profile_image_url,twitter_url,alias,created_at,updated_at
uuid,,,,,,,,,,,,,,,,,,,,,
f5bb580f-d655-cf3f-9ade-eca5b3f2719f,"Android, Apple - iOS, Blackberry, Windows Phon...",VilarikA,company,/organization/vilarika,vilarika.com.br,http://vilarika.com.br/,BRA,NaN,Rio de Janeiro,Belo Horizonte,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/vilarika,https://www.crunchbase.com/organization/vilari...,http://public.crunchbase.com/t_api_images/v141...,NaN,NaN,2014-12-11 06:46:05,2016-09-07 00:03:51.67913
00000aa4-ba42-9b68-a9c3-040c9f3bf9b9,Formel D GmbH is a automotive manufacturer and...,Formel D GmbH,company,/organization/formel-d-gmbh,formeld.com,http://www.formeld.com,DEU,NaN,DEU - Other,Troisdorf,...,+49 2241 9960,https://www.facebook.com/formeld,https://www.linkedin.com/company/formel-d-group,https://www.crunchbase.com/organization/formel...,https://www.crunchbase.com/organization/formel...,http://public.crunchbase.com/t_api_images/v148...,https://www.twitter.com/formeld_es,NaN,2016-06-01 06:58:37.692725,2017-07-18 07:22:19.15864
f5bd38d1-7719-2935-fb6c-defae39f5b93,[iNFoGooL](http://infogool.com) - The Informat...,infogool,company,/organization/infogool,infogool.com,http://infogool.com,NaN,NaN,NaN,NaN,...,8802393009,http://www.facebook.com/infogool,NaN,https://www.crunchbase.com/organization/infogool,https://www.crunchbase.com/organization/infogo...,http://public.crunchbase.com/t_api_images/v139...,https://www.twitter.com/infogool,NaN,2014-04-12 00:10:12,2016-09-08 22:02:05.585875
773adc18-132a-937d-8841-4f833e64dd56,The Jinfeng Gold Mine is an combined open pit ...,Jinfeng Mine,company,/organization/jinfeng-mine,NaN,NaN,CHN,NaN,NaN,NaN,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/jinfen...,NaN,NaN,NaN,NaN,2016-12-07 06:43:06.955155,2016-12-07 06:45:09.14727
f5bdd48a-a09f-2f4c-bd71-4c4207c7731b,Peardoc offers online tools to convert HTML to...,Peardoc Solutions,company,/organization/peardoc-solutions,peardoc.com,http://www.peardoc.com,IND,NaN,Chennai,Chennai,...,NaN,NaN,NaN,https://www.crunchbase.com/organization/peardo...,https://www.crunchbase.com/organization/peardo...,http://public.crunchbase.com/t_api_images/v140...,https://www.twitter.com/pear_doc,NaN,2014-07-17 10:06:03,2016-03-08 02:56:01.230586


In [202]:
len(result1)

NameError: name 'result1' is not defined

In [107]:
# Constuct a list of companies that have descriptions
#d_companies_list = list(result1.loc[:, "company_name"])
#d_companies_list = [company.lower() for company in d_companies_list]
#len(d_companies_list)

In [126]:
# Construct a list of companies from dataset folder
f_companies_list = os.listdir(os.path.join(basedir, "News_1"))
# Get rid of ".json" extension and apply uppercase
f_companies_list = [re.sub(".json", "", company) for company in f_companies_list]
print(len(f_companies_list))

20249


In [127]:
# Get rid of companies with names containing anything else than latin letters (as well as containing "_" separating multiple words in names)
f_companies_list = [company.lower() for company in f_companies_list if all(letter in string.ascii_letters for letter in company)]
print(len(f_companies_list))

# Get rid of companies whose full names consist of one single word from english dictionary
f_companies_list = [company for company in f_companies_list if not d.check(company.lower())]
print(len(f_companies_list))

12609
11385


In [125]:
def word_exists(word, apiKey=apiKey, apiUrl='http://api.wordnik.com/v4'):

    # Construct the API URL for a random-word query
    api_url = "{baseurl}/words.json/search/{word}".format(
        baseurl=apiUrl,
        word=word
    )

    parameters = {
        'api_key': apiKey,
    }

    # Perform the query and store the HTTP response object
    response = requests.get(api_url, params=parameters)

    # Convert the response content to a list
    # NOTE: The content is initially returned as a byte string
    word_object = json.loads(response.content)

    # Get the number word's instances in the dictionary
    if word_object:
        if word_object['searchResults'][0]['count'] > 0:
            result = True
        else:
            result = False
    else:
        result = False

    # Return the word as a string
    return result

# Same as the previous step (looking for english words) but with Wordnik API
apiUrl = 'http://api.wordnik.com/v4'
apiKey = 'b0a856e4cbe5c4e3f110d0527b901b7064f22a1c09e153547'
# Extract the list words from Wordnik that DO exist
wordnik_companies_list = [company for company in f_companies_list if word_exists(company)]
print(len(wordnik_companies_list))

# These words DO exist in Wordnik dictionary
print((wordnik_companies_list))

['accend', 'acton', 'aerospike', 'afterschool', 'ais', 'alation', 'alector', 'alignable', 'amberjack', 'amicus', 'ananas', 'ansible', 'anyroad', 'aptera', 'aruspex', 'asana', 'attask', 'auris', 'authorly', 'aviso', 'ayr', 'backchat', 'backdoor', 'backtrace', 'bannerman', 'bastille', 'beme', 'bento', 'bevvy', 'biome', 'biscotti', 'bittorrent', 'blesh', 'bonafide', 'bonobos', 'bookbag', 'boombox', 'booyah', 'bottlenose', 'bowery', 'boxfish', 'brit', 'browster', 'carbonite', 'cardio', 'cartesian', 'cartogram', 'celly', 'centro', 'chargeback', 'chockstone', 'chronicity', 'clickable', 'cloze', 'containership', 'contentful', 'convo', 'copromote', 'courseload', 'courseloads', 'cringle', 'crossfader', 'cuculus', 'dekko', 'dextro', 'digby', 'doo', 'doozie', 'doppelganger', 'driverside', 'dropoff', 'dropship', 'duetto', 'eco', 'elastica', 'elsen', 'emma', 'endorphin', 'endplay', 'enow', 'entrada', 'epiphyte', 'eponym', 'eps', 'ess', 'everly', 'evolv', 'evolver', 'exogenesis', 'eyespot', 'fab', '

In [10]:
# Save for later use 
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open(os.path.join(basedir, 'temp_data','wordnik_companies_list.txt'), 'wb') as fp:
    pickle.dump(wordnik_companies_list, fp)

In [11]:
with open (os.path.join(basedir, 'temp_data', 'wordnik_companies_list.txt'), 'rb') as fp:
    wordnik_companies_list = pickle.load(fp)

In [104]:
# Tries to separate a name string into multiple words and if succeeds gets rid of the company (a lot of false positives)
# companies_list = [company for company in companies_list if len(wordninja.split(company.lower())) < 2]
# print(len(companies_list))

11385
11385
1


In [132]:
f_companies_list = [e for e in f_companies_list if e not in set(wordnik_companies_list)]
print(len(f_companies_list))
print(f_companies_list[0:100])

11018
['aaptiv', 'abaris', 'abbeypost', 'abcmob', 'abeo', 'abililife', 'abilto', 'abine', 'abiobot', 'ablexis', 'ablio', 'abodo', 'abogen', 'abom', 'aboutlife', 'aboutme', 'aboutone', 'aboutusorg', 'abra', 'absci', 'absmaterials', 'abusix', 'abvitro', 'abyrx', 'academiaedu', 'academixdirect', 'acadiasoft', 'accelera', 'accelereach', 'accelergy', 'accelgolf', 'accellos', 'accelo', 'accelops', 'acceptd', 'accera', 'accertify', 'accesssportsmediacom', 'acclaimd', 'acclarent', 'accountnow', 'accredible', 'accreon', 'accuitis', 'acculitx', 'accumen', 'accuradio', 'accuvein', 'accuwater', 'aceable', 'acebotai', 'acertiv', 'achaogen', 'achieveit', 'acompli', 'acopio', 'acousticeye', 'acquaintable', 'acquia', 'acrinta', 'acrisure', 'acrobatiq', 'acronis', 'acsian', 'actacell', 'actifio', 'actionality', 'actionsprout', 'actionx', 'activaero', 'actived', 'activegrid', 'activehours', 'activepath', 'activeprotective', 'activerain', 'activiter', 'activityhero', 'actmd', 'actualmeds', 'actuatedmedic

In [133]:
# Prepare the data frame of companies descriptions to be used in final data frame with articles
d_companies_df = result1
d_companies_df.loc[:,'company_name'] = d_companies_df.loc[:,'company_name'].str.lower()
#d_companies_df = d_companies_df[d_companies_df['company_name'].str.lower().isin(d_companies_list)]
#d_companies_df = d_companies_df[d_companies_df['company_name'].isin(d_companies_list)]
d_companies_df = d_companies_df.reset_index().set_index(keys='company_name')#, verify_integrity=True)



# Delete the companies which have the same name
d_companies_df = d_companies_df[~d_companies_df.index.duplicated(keep=False)]


d_companies_df = d_companies_df[['uuid', 'description']]
len(d_companies_df)

345890

In [134]:
# Delete companies outside or dataset of articles
companies_df = d_companies_df[d_companies_df.index.isin(f_companies_list)]
companies_list = companies_df.index.values
print(len(companies_df))

9658


In [6]:
# Save for later use with keywords extraction
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open(os.path.join(basedir, 'temp_data', 'companies_df'), 'wb') as fp:
    pickle.dump(companies_df, fp)

In [25]:
with open (os.path.join(basedir, 'temp_data', 'companies_df'), 'rb') as fp:
    companies_df = pickle.load(fp)
companies_list = companies_df.index.values
print(len(companies_df))
companies_df.head()


9658


,uuid,description
company_name,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur..."


In [32]:
### TWO WAYS TO EXTRACT KEYWORDS ###




# With PKE Package - TopicRank
# initialize keyphrase extraction model, here TopicRank
import pke

# With EmbedRank
cwd = os.getcwd()
try:
    os.chdir(os.path.join(os.path.dirname(cwd), path_to_embedrank_repo))
except FileNotFoundError:
    try:
        os.chdir(path_to_embedrank_repo)
    except FileNotFoundError:
        os.chdir(cwd) 
import launch
print(os.getcwd())
embedding_distributor = launch.load_local_embedding_distributor('en')
pos_tagger = launch.load_local_pos_tagger('en')

/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction


/home/tie-server/anaconda3/lib/python3.6/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [203]:
k_companies_df.head()

,uuid,description,kp_topicrank,kp_embedrank
company_name,,,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog...","[market, technical innovation, company, synthe...","[world class technology, natural tissue repair..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[classâ€ products, hemodialysis access graft,...","[human clinical trials, hemodialysis access gr..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[systems, development stage company, company, ...","[blood alcohol detection system, construction ..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...,"[thrombovision, medical diagnosis, biomedical ...","[medical diagnosis, critical cardiovascular ri..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[access policies, fortress capsule, manages, s...","[access policies, fortress capsule, virtual wo..."


In [ ]:
counter = 0
k_companies_df = companies_df.copy()

for index, row in companies_df.iterrows():
    
    print(str(datetime.datetime.now()) + " " + index)    
    
    counter +=1
    #if counter >3:
    #    break
    
    extractor = pke.unsupervised.TopicRank()
    # load the content of the document, here document is expected to be in raw
    # format (i.e. a simple string of text) and preprocessing is carried out using nltk
    s = row["description"]
    extractor.read_text(s)

    # keyphrase candidate selection, in the case of TopicRank: sequences of nouns
    # and adjectives
    extractor.candidate_selection()

    try:
        # candidate weighting, in the case of TopicRank: using a random walk algorithm
        extractor.candidate_weighting()
        # N-best selection, keyphrases contains the 10 highest scored candidates as
        # (keyphrase, score) tuples
        keyphrases = extractor.get_n_best(n=5, stemming=False)
        keyphrases = sorted(keyphrases, key=lambda x: x[1], reverse=True)

        k_companies_df.loc[index, 'kp_topicrank'] = str(keyphrases)
    
    except ValueError:
        print("TOPIC_RANK: Description for company " + index + 
              " (" + row['uuid'] + ")" +
              " is too short. Description: " + index)
        continue
    
    kp1 = launch.extract_keyphrases(embedding_distributor, pos_tagger, s, 5, 'en') 
    if None in kp1:
        print("EMBED_RANK: Description for company " + index + 
                  " (" + row['uuid'] + ")" +
                  " is too short. Description: " + index)
        continue    
    k_companies_df.loc[index, 'kp_embedrank'] = str(kp1)
    
k_companies_df.dropna(subset=["kp_topicrank"], inplace=True)
k_companies_df.dropna(subset=["kp_embedrank"], inplace=True)

2018-07-23 21:29:15.836455nerites
2018-07-23 21:29:18.243740biovascular
2018-07-23 21:29:20.784224transbiotec
2018-07-23 21:29:23.402018thrombovision
2018-07-23 21:29:25.952495fortressware
2018-07-23 21:29:28.504126songfor
2018-07-23 21:29:31.250156buildpulse
2018-07-23 21:29:33.834544gogograndparent
2018-07-23 21:29:36.369559changenexus


/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:44: RuntimeWarning: invalid value encountered in true_divide
  0.5 + (sim_between_norm - np.nanmean(sim_between_norm, axis=0)) / np.nanstd(sim_between_norm, axis=0)


2018-07-23 21:29:39.063988connectbeam
2018-07-23 21:29:41.654636connoshoer
2018-07-23 21:29:44.185940multispan
2018-07-23 21:29:46.750198oculusit
2018-07-23 21:29:49.370381loctote
2018-07-23 21:29:52.036354freewavz
2018-07-23 21:29:54.547487fabtask
2018-07-23 21:29:57.189848fantrail
2018-07-23 21:29:59.740534krauttools
2018-07-23 21:30:02.293665prenova
2018-07-23 21:30:04.920419freepath
2018-07-23 21:30:07.409902naturtrip
2018-07-23 21:30:09.895554kabongo
2018-07-23 21:30:12.374737crystalplex
2018-07-23 21:30:14.973391bindhq
2018-07-23 21:30:17.559213phonio
2018-07-23 21:30:20.232466invup
2018-07-23 21:30:22.750827zynbit
2018-07-23 21:30:25.477715statementone
2018-07-23 21:30:28.001336neighbormd
2018-07-23 21:30:30.591867xbux
2018-07-23 21:30:33.145078helicomm
2018-07-23 21:30:35.717279arrister
2018-07-23 21:30:38.251769remotereps
2018-07-23 21:30:40.843788artvenue
2018-07-23 21:30:43.321783doblet
2018-07-23 21:30:45.849859duxplore
2018-07-23 21:30:48.558664sereniti
2018-07-23 21:30:51

/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:37: RuntimeWarning: invalid value encountered in true_divide
  doc_sim_norm = 0.5 + (doc_sim_norm - np.average(doc_sim_norm)) / np.std(doc_sim_norm)
/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:42: RuntimeWarning: All-NaN slice encountered
  sim_between_norm = sim_between/np.nanmax(sim_between, axis=0)
/media/tie-server/Workspace/Jens_Artemiy/ai-research-keyphrase-extraction/swisscom_ai/research_keyphrase/model/method.py:44: RuntimeWarning: Mean of empty slice
  0.5 + (sim_between_norm - np.nanmean(sim_between_norm, axis=0)) / np.nanstd(sim_between_norm, axis=0)
/home/tie-server/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1427: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


2018-07-23 21:33:51.036289eainvest
2018-07-23 21:33:53.537516cleanfish
2018-07-23 21:33:56.217629preisbock
2018-07-23 21:33:58.824156caserev
2018-07-23 21:34:01.394542screenmedix
2018-07-23 21:34:03.942409primedic
2018-07-23 21:34:06.501635treatful
2018-07-23 21:34:09.068809calyxo
2018-07-23 21:34:11.767354workmob
2018-07-23 21:34:14.389074globein
2018-07-23 21:34:16.926770ammasso
2018-07-23 21:34:19.514855empliant
2018-07-23 21:34:22.127885nesteggguru
2018-07-23 21:34:24.674372aerobo
2018-07-23 21:34:27.215945fortiusone
2018-07-23 21:34:29.718377gigacrete
2018-07-23 21:34:32.309962alverix
2018-07-23 21:34:34.829370informous
2018-07-23 21:34:37.315183inviragen
2018-07-23 21:34:39.886843polimetrix
2018-07-23 21:34:42.632215proteopure
2018-07-23 21:34:45.097752ocdesk
2018-07-23 21:34:47.800056biotrackthc
2018-07-23 21:34:50.420718angiogenex
2018-07-23 21:34:52.930081hyperpot
2018-07-23 21:34:55.497267siut
2018-07-23 21:34:58.092857gonoogie
2018-07-23 21:35:00.586139mailexpress
2018-07-23

...

2018-07-24 02:00:34.860058solmentum
2018-07-24 02:00:37.545885goodworld
2018-07-24 02:00:40.111066sampa
2018-07-24 02:00:42.844322havemyshift
2018-07-24 02:00:45.570425minitime
2018-07-24 02:00:48.161426jamplify
2018-07-24 02:00:50.726898judicata
2018-07-24 02:00:53.261623homeunion
2018-07-24 02:00:55.739679mindmixer
2018-07-24 02:00:58.396594brightlot
2018-07-24 02:01:00.913320codersclan
2018-07-24 02:01:03.487789doobop
2018-07-24 02:01:06.193499kustomnote
2018-07-24 02:01:08.698418cirrosecure
2018-07-24 02:01:11.259025murfie
2018-07-24 02:01:13.794549prepflash
2018-07-24 02:01:16.328620prepify
2018-07-24 02:01:18.824248callsfreecalls
2018-07-24 02:01:21.471230feedtrace
2018-07-24 02:01:24.020435elucify
2018-07-24 02:01:26.557895pilldrops
2018-07-24 02:01:29.267435clinicast
2018-07-24 02:01:31.973760retrotope
2018-07-24 02:01:34.494797marxent
2018-07-24 02:01:37.083687lumenergi
2018-07-24 02:01:39.687373wellinks
2018-07-24 02:01:42.224713bunndle
2018-07-24 02:01:44.839447mdxhealth
201

2018-07-24 02:10:43.042309uavionix
2018-07-24 02:10:45.617074viraltag
2018-07-24 02:10:48.234935qualvu
2018-07-24 02:10:50.850845sickweather
2018-07-24 02:10:53.343755frs
2018-07-24 02:10:56.146787teraki
2018-07-24 02:10:58.661613bolstra
2018-07-24 02:11:01.279780eyenalyze
2018-07-24 02:11:04.033334triplebyte
2018-07-24 02:11:06.514086datacamp
2018-07-24 02:11:09.066301fixetude
2018-07-24 02:11:11.373752qwil
2018-07-24 02:11:13.931974pacatio
2018-07-24 02:11:16.486468nousdecor
2018-07-24 02:11:19.116640plumtv
2018-07-24 02:11:21.605633clearflow
2018-07-24 02:11:24.334660stylezen
2018-07-24 02:11:26.927112comvibe
2018-07-24 02:11:29.571593stayful
2018-07-24 02:11:32.108694bellabeat
2018-07-24 02:11:34.652054readfy
2018-07-24 02:11:37.386248wehostels
2018-07-24 02:11:39.912684deskgod
2018-07-24 02:11:43.132466brandshield
2018-07-24 02:11:45.927842evogen
2018-07-24 02:11:48.489782madefire
2018-07-24 02:11:51.101987remesh
2018-07-24 02:11:53.601915lawpal
2018-07-24 02:11:56.134658gopuff
20

2018-07-24 02:20:53.392470umake
2018-07-24 02:20:55.924474prosky
2018-07-24 02:20:58.441400rivs
2018-07-24 02:21:00.928527engrade
2018-07-24 02:21:03.490585readz
2018-07-24 02:21:06.022569greenphire
2018-07-24 02:21:08.592704colabo
2018-07-24 02:21:11.150027iggbo
2018-07-24 02:21:13.763169viewlift
2018-07-24 02:21:16.465409trovix
2018-07-24 02:21:19.254606courbanize
2018-07-24 02:21:21.487818focalcast
2018-07-24 02:21:24.025851lookcast
2018-07-24 02:21:26.623597curtran
2018-07-24 02:21:29.200670gdgt
2018-07-24 02:21:31.898322crashlytics
2018-07-24 02:21:34.477959keywee
2018-07-24 02:21:37.083105powerlytics
2018-07-24 02:21:39.598405textpower
2018-07-24 02:21:42.267583tipbit
2018-07-24 02:21:44.777136fattmerchant
2018-07-24 02:21:47.413122nascentric
2018-07-24 02:21:49.975684visarity
2018-07-24 02:21:52.520824neoantigenics
2018-07-24 02:21:55.112168synerchip
2018-07-24 02:21:57.765381didlog
2018-07-24 02:22:00.361924ecomdash
2018-07-24 02:22:02.888350fhoosh
2018-07-24 02:22:05.491826yep

2018-07-24 02:30:33.808511snappydata
2018-07-24 02:30:36.318374useready
2018-07-24 02:30:38.968675frontstream
2018-07-24 02:30:41.573506foneshow
2018-07-24 02:30:44.195271beegit
2018-07-24 02:30:46.812633naverus
2018-07-24 02:30:49.444845vaycayhero
2018-07-24 02:30:52.019627avogy
2018-07-24 02:30:54.537674vestwell
2018-07-24 02:30:57.064442eagerpanda
2018-07-24 02:30:59.704715sipsynergy
2018-07-24 02:31:02.244111eyefluence
2018-07-24 02:31:04.930046actualmeds
2018-07-24 02:31:07.565377nestiny
2018-07-24 02:31:10.185717showkit
2018-07-24 02:31:12.910725songcat
2018-07-24 02:31:15.431286urtak
2018-07-24 02:31:18.005078labnow
2018-07-24 02:31:20.636423noovo
2018-07-24 02:31:23.200935akitabox
2018-07-24 02:31:25.711845crexi
2018-07-24 02:31:28.294609oomnitza
2018-07-24 02:31:30.936039emms
TOPIC_RANK: Description for company emms (4eeacc81-59cd-5d60-e32b-313d6d0e9d9a) is too short. Description: emms
2018-07-24 02:31:30.939340healbe
2018-07-24 02:31:33.661989fabbeo
2018-07-24 02:31:36.360975

2018-07-24 02:40:37.080394flowcardia
2018-07-24 02:40:39.718202peachworks
2018-07-24 02:40:42.298971paydivvy
2018-07-24 02:40:44.981314kitchenbowl
2018-07-24 02:40:47.500778moodnode
2018-07-24 02:40:50.127344repairogen
2018-07-24 02:40:52.803433youtern
2018-07-24 02:40:55.521505homewellness
2018-07-24 02:40:58.170008auctiondrop
2018-07-24 02:41:00.743833xtent
2018-07-24 02:41:03.295275diy
2018-07-24 02:41:05.970071watchsend
2018-07-24 02:41:08.463869shotpoint
2018-07-24 02:41:11.006017futureleague
2018-07-24 02:41:13.579879intronet
2018-07-24 02:41:16.233641moneythink
2018-07-24 02:41:18.757143triplingo
2018-07-24 02:41:21.498256bindo
2018-07-24 02:41:24.035827cardtapp
2018-07-24 02:41:26.599885tivity
2018-07-24 02:41:29.156892zeepro
2018-07-24 02:41:31.599152perceivant
2018-07-24 02:41:34.089237soley
2018-07-24 02:41:36.404635levlr
2018-07-24 02:41:38.845804pacinian
2018-07-24 02:41:41.520683ampsy
2018-07-24 02:41:44.211396smartfile
2018-07-24 02:41:46.959300accelera
2018-07-24 02:41:

In [37]:

print(str(len(companies_df) - len(k_companies_df)) + " companies did not have sufficient length of descriptions")
k_companies_df.tail()

23 companies did not have sufficient length of descriptions


,uuid,description
company_name,,
hooja,ff71401a-f5b0-1dd8-2b72-4a9e5d6ecba0,Hooja is a company focused on the mobile realm...
baseclick,d140620f-09bb-dd33-baaa-ecffff1d8bc6,baseclick GmbH is a start-up company funded an...
biodetego,fef487a3-2505-e176-7bdc-d82db18525dc,"BioDetego is developing, VASPfore, a new bioma..."
socialthreader,ffe12892-05c4-e8e4-d728-d9b54e7b3aeb,SocialThreader increases effectiveness of digi...
findally,cc693948-abf3-e3dc-f845-3e38ac718aab,Findally is an image-based product engine that...


In [90]:
k_with_confidences_companies_df = k_companies_df.copy()

In [129]:
counter = 0
for company, row in k_with_confidences_companies_df.iterrows():
    counter = counter + 1
    #if counter > 2:
    #    break
    print(str(datetime.datetime.now()) + " " + company)
    k_companies_df.loc[company,"kp_embedrank"] = ast.literal_eval(row["kp_embedrank"])[0]
    k_companies_df.loc[company,"kp_topicrank"] = [kp[0] for kp in ast.literal_eval(row["kp_topicrank"])]
k_companies_df.head()
#print(counter)

2018-07-24 13:33:17.692272 nerites
2018-07-24 13:33:17.703168 biovascular
2018-07-24 13:33:17.709126 transbiotec
2018-07-24 13:33:17.715054 thrombovision
2018-07-24 13:33:17.721062 fortressware
2018-07-24 13:33:17.727047 songfor
2018-07-24 13:33:17.733073 buildpulse
2018-07-24 13:33:17.739109 gogograndparent
2018-07-24 13:33:17.745136 changenexus
2018-07-24 13:33:17.751109 connectbeam
2018-07-24 13:33:17.757201 connoshoer
2018-07-24 13:33:17.770460 multispan
2018-07-24 13:33:17.779401 oculusit
2018-07-24 13:33:17.785200 loctote
2018-07-24 13:33:17.790971 freewavz
2018-07-24 13:33:17.796732 fabtask
2018-07-24 13:33:17.802476 fantrail
2018-07-24 13:33:17.808235 krauttools
2018-07-24 13:33:17.831315 prenova
2018-07-24 13:33:17.837725 freepath
2018-07-24 13:33:17.843458 naturtrip
2018-07-24 13:33:17.849221 kabongo
2018-07-24 13:33:17.854938 crystalplex
2018-07-24 13:33:17.862051 bindhq
2018-07-24 13:33:17.867844 phonio
2018-07-24 13:33:17.873634 invup
2018-07-24 13:33:17.879380 zynbit
2018

2018-07-24 13:33:19.345021 riide
2018-07-24 13:33:19.350099 procom
2018-07-24 13:33:19.354874 arkivio
2018-07-24 13:33:19.359678 technospin
2018-07-24 13:33:19.364606 optionease
2018-07-24 13:33:19.369363 groupstream
2018-07-24 13:33:19.374112 musicplayr
2018-07-24 13:33:19.378857 apexigen
2018-07-24 13:33:19.384138 latherm
2018-07-24 13:33:19.389010 peoplug
2018-07-24 13:33:19.400839 brandshub
2018-07-24 13:33:19.406126 znode
2018-07-24 13:33:19.410870 wemail
2018-07-24 13:33:19.415726 solannex
2018-07-24 13:33:19.420628 akabotics
2018-07-24 13:33:19.425423 fundability
2018-07-24 13:33:19.430230 potguide
2018-07-24 13:33:19.434933 actionality
2018-07-24 13:33:19.439806 nanopack
2018-07-24 13:33:19.444539 provasculon
2018-07-24 13:33:19.449429 therative
2018-07-24 13:33:19.454246 fatpipe
2018-07-24 13:33:19.458987 ipawn
2018-07-24 13:33:19.463809 aitainment
2018-07-24 13:33:19.468643 mentinova
2018-07-24 13:33:19.473513 taperecorder
2018-07-24 13:33:19.478334 envirokure
2018-07-24 13:3

2018-07-24 13:33:20.560376 corimmun
2018-07-24 13:33:20.565330 cariocas
2018-07-24 13:33:20.570271 icrimefighter
2018-07-24 13:33:20.575261 sketchme
2018-07-24 13:33:20.580184 cloudcom
2018-07-24 13:33:20.585120 everlater
2018-07-24 13:33:20.590051 brightcom
2018-07-24 13:33:20.594977 roomations
2018-07-24 13:33:20.599929 gigalogix
2018-07-24 13:33:20.604873 kyoger
2018-07-24 13:33:20.609814 linkedin
2018-07-24 13:33:20.614751 groupon
2018-07-24 13:33:20.620241 zynga
2018-07-24 13:33:20.625187 airbnb
2018-07-24 13:33:20.630137 docusign
2018-07-24 13:33:20.635067 techstars
2018-07-24 13:33:20.640006 twilio
2018-07-24 13:33:20.644959 youtube
2018-07-24 13:33:20.649903 mongodb
2018-07-24 13:33:20.654841 pinterest
2018-07-24 13:33:20.660089 skillz
2018-07-24 13:33:20.665035 cloudera
2018-07-24 13:33:20.669953 dropbox
2018-07-24 13:33:20.674872 crunchbase
2018-07-24 13:33:20.679773 marketo
2018-07-24 13:33:20.684708 buzzfeed
2018-07-24 13:33:20.689617 github
2018-07-24 13:33:20.694496 hubsp

2018-07-24 13:33:22.558624 icontact
2018-07-24 13:33:22.563713 educents
2018-07-24 13:33:22.568664 tealium
2018-07-24 13:33:22.573884 demandware
2018-07-24 13:33:22.581039 carecloud
2018-07-24 13:33:22.587589 dataxu
2018-07-24 13:33:22.594184 zenoss
2018-07-24 13:33:22.600713 adaptly
2018-07-24 13:33:22.607268 tubemogul
2018-07-24 13:33:22.613891 aglocal
2018-07-24 13:33:22.620455 chartbeat
2018-07-24 13:33:22.627060 apperian
2018-07-24 13:33:22.633641 scribd
2018-07-24 13:33:22.640139 pixable
2018-07-24 13:33:22.646173 agilone
2018-07-24 13:33:22.652440 gainsight
2018-07-24 13:33:22.658975 avalara
2018-07-24 13:33:22.665550 younoodle
2018-07-24 13:33:22.671259 seesmic
2018-07-24 13:33:22.677770 shyp
2018-07-24 13:33:22.684283 mobileiron
2018-07-24 13:33:22.690881 kixeye
2018-07-24 13:33:22.697539 zulily
2018-07-24 13:33:22.704142 carepredict
2018-07-24 13:33:22.709839 kickstarter
2018-07-24 13:33:22.717319 spreadshirt
2018-07-24 13:33:22.723926 swrve
2018-07-24 13:33:22.730495 cipherc

2018-07-24 13:33:23.896544 convercent
2018-07-24 13:33:23.901869 adchemy
2018-07-24 13:33:23.906598 bloomreach
2018-07-24 13:33:23.911324 gymgroups
2018-07-24 13:33:23.916759 kaminario
2018-07-24 13:33:23.921602 veniam
2018-07-24 13:33:23.926394 edsurge
2018-07-24 13:33:23.931131 roundbox
2018-07-24 13:33:23.935895 morphlabs
2018-07-24 13:33:23.940702 affectiva
2018-07-24 13:33:23.945595 adready
2018-07-24 13:33:23.950478 meegenius
2018-07-24 13:33:23.955280 couchsurfing
2018-07-24 13:33:23.960075 nowthis
2018-07-24 13:33:23.964920 outsystems
2018-07-24 13:33:23.969775 tapinfluence
2018-07-24 13:33:23.974494 venafi
2018-07-24 13:33:23.979335 shopadvisor
2018-07-24 13:33:23.984116 veoh
2018-07-24 13:33:23.988954 waze
2018-07-24 13:33:23.993765 connectifier
2018-07-24 13:33:23.998558 ignitionone
2018-07-24 13:33:24.003378 metromile
2018-07-24 13:33:24.008186 homejoy
2018-07-24 13:33:24.012987 airware
2018-07-24 13:33:24.017791 qumulo
2018-07-24 13:33:24.022702 newforma
2018-07-24 13:33:2

...

2018-07-24 13:34:02.677320 ibeatyou
2018-07-24 13:34:02.682301 haload
2018-07-24 13:34:02.687524 hivemapper
2018-07-24 13:34:02.692260 adviceiq
2018-07-24 13:34:02.696997 ceterus
2018-07-24 13:34:02.701743 sendia
2018-07-24 13:34:02.706477 eegoes
2018-07-24 13:34:02.711170 adtriba
2018-07-24 13:34:02.715907 pressmatrix
2018-07-24 13:34:02.720671 sidestripe
2018-07-24 13:34:02.725508 xtuple
2018-07-24 13:34:02.730382 toposens
2018-07-24 13:34:02.735129 iforem
2018-07-24 13:34:02.739871 algenetix
2018-07-24 13:34:02.744627 lorious
2018-07-24 13:34:02.749462 lendamend
2018-07-24 13:34:02.754201 rvita
2018-07-24 13:34:02.758940 pathsensors
2018-07-24 13:34:02.763660 intelinair
2018-07-24 13:34:02.768392 continuent
2018-07-24 13:34:02.773180 egym
2018-07-24 13:34:02.777975 cargoguard
2018-07-24 13:34:02.782712 actacell
2018-07-24 13:34:02.787442 multivoice
2018-07-24 13:34:02.792217 classiphix
2018-07-24 13:34:02.797058 sicortex
2018-07-24 13:34:02.801894 givenext
2018-07-24 13:34:02.806670

2018-07-24 13:34:03.918818 lepow
2018-07-24 13:34:03.923945 hdmessaging
2018-07-24 13:34:03.928733 collegezen
2018-07-24 13:34:03.933492 badabroad
2018-07-24 13:34:03.938235 prisyna
2018-07-24 13:34:03.942987 dimensionalmechanics
2018-07-24 13:34:03.947725 putney
2018-07-24 13:34:03.952463 artaic
2018-07-24 13:34:03.957279 venuenext
2018-07-24 13:34:03.962011 apptrigger
2018-07-24 13:34:03.966770 seclarity
2018-07-24 13:34:03.971561 knowlent
2018-07-24 13:34:03.976384 sumomi
2018-07-24 13:34:03.981143 giiv
2018-07-24 13:34:03.985870 pylba
2018-07-24 13:34:03.990614 baremetrics
2018-07-24 13:34:03.995408 sochat
2018-07-24 13:34:04.000144 lockr
2018-07-24 13:34:04.004876 mobappcreator
2018-07-24 13:34:04.009621 ipling
2018-07-24 13:34:04.014364 biisafe
2018-07-24 13:34:04.019096 dewmobile
2018-07-24 13:34:04.023832 peerfit
2018-07-24 13:34:04.028585 greendimes
2018-07-24 13:34:04.033313 trackme
2018-07-24 13:34:04.038057 tendertree
2018-07-24 13:34:04.042827 cytosolv
2018-07-24 13:34:04.

2018-07-24 13:34:05.165212 returnly
2018-07-24 13:34:05.170252 skoovy
2018-07-24 13:34:05.174986 bioscale
2018-07-24 13:34:05.179723 arthromeda
2018-07-24 13:34:05.184462 safeview
2018-07-24 13:34:05.189222 gotit
2018-07-24 13:34:05.193953 bontera
2018-07-24 13:34:05.198690 cotera
2018-07-24 13:34:05.203465 adeze
2018-07-24 13:34:05.208195 ibabybox
2018-07-24 13:34:05.212936 diningcircle
2018-07-24 13:34:05.217662 stukent
2018-07-24 13:34:05.222386 dabkick
2018-07-24 13:34:05.227201 worksurfers
2018-07-24 13:34:05.232127 aimwith
2018-07-24 13:34:05.236872 igor
2018-07-24 13:34:05.241665 kluster
2018-07-24 13:34:05.246422 estorian
2018-07-24 13:34:05.251155 kashless
2018-07-24 13:34:05.255905 dermamedics
2018-07-24 13:34:05.260636 quantiacs
2018-07-24 13:34:05.265365 scalyr
2018-07-24 13:34:05.270094 trackin
2018-07-24 13:34:05.274834 classiqs
2018-07-24 13:34:05.279570 wireover
2018-07-24 13:34:05.284311 contix
2018-07-24 13:34:05.289058 moblico
2018-07-24 13:34:05.293798 healthentic
2

2018-07-24 13:34:06.405564 vimty
2018-07-24 13:34:06.410701 govpredict
2018-07-24 13:34:06.415685 rapidrabbit
2018-07-24 13:34:06.420584 payteller
2018-07-24 13:34:06.425496 dueprops
2018-07-24 13:34:06.430429 libriloop
2018-07-24 13:34:06.435396 cannuflow
2018-07-24 13:34:06.440331 apicloud
2018-07-24 13:34:06.445288 loopool
2018-07-24 13:34:06.450201 sencera
2018-07-24 13:34:06.455109 meetmetix
2018-07-24 13:34:06.460034 innometrix
2018-07-24 13:34:06.465034 cittio
2018-07-24 13:34:06.470013 dilitronics
2018-07-24 13:34:06.474987 mydrobe
2018-07-24 13:34:06.480007 vittamed
2018-07-24 13:34:06.484918 cartified
2018-07-24 13:34:06.489859 ofuz
2018-07-24 13:34:06.494827 spideroak
2018-07-24 13:34:06.499750 valvexchange
2018-07-24 13:34:06.504709 techniscan
2018-07-24 13:34:06.509628 consortiex
2018-07-24 13:34:06.514543 synup
2018-07-24 13:34:06.519456 enloopp
2018-07-24 13:34:06.524397 coveritlive
2018-07-24 13:34:06.529318 ntirety
2018-07-24 13:34:06.534229 stryd
2018-07-24 13:34:06.5

,uuid,description,kp_topicrank,kp_embedrank
company_name,,,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog...","[market, technical innovation, company, synthe...","[world class technology, natural tissue repair..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[classâ€ products, hemodialysis access graft,...","[human clinical trials, hemodialysis access gr..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[systems, development stage company, company, ...","[blood alcohol detection system, construction ..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...,"[thrombovision, medical diagnosis, biomedical ...","[medical diagnosis, critical cardiovascular ri..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[access policies, fortress capsule, manages, s...","[access policies, fortress capsule, virtual wo..."


In [134]:
# Save entire dataframe with keyphrases and confidences
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open(os.path.join(basedir, 'temp_data', 'k_with_confidences_companies_df'), 'wb') as fp:
    pickle.dump(k_with_confidences_companies_df, fp)

In [50]:
# Save entire dataframe with keyphrases
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open(os.path.join(basedir, 'temp_data', 'k_companies_df'), 'wb') as fp:
    pickle.dump(k_companies_df, fp)

In [88]:
with open (os.path.join(basedir, 'temp_data', 'k_companies_df'), 'rb') as fp:
    k_companies_df = pickle.load(fp)
k_companies_list = k_companies_df.index.values
k_companies_df.head()

,uuid,description,kp_topicrank,kp_embedrank
company_name,,,,
nerites,f5c05e57-d563-db35-8210-6f26d6a250d2,"At Nerites, they combine world class technolog...","[('market', 0.09826419107155268), ('technical ...","(['world class technology', 'natural tissue re..."
biovascular,7539d769-643c-6c26-5339-ec52bd5b81c2,BioVascular is a clinical stage privately held...,"[('classâ€\x9d products', 0.09608849550420087)...","(['human clinical trials', 'hemodialysis acces..."
transbiotec,f5e29c35-a53e-635d-ce74-4013c11efec9,"TransBiotec, a development stage company, focu...","[('systems', 0.09053148884251386), ('developme...","(['blood alcohol detection system', 'construct..."
thrombovision,7758702e-6321-ee55-16d9-11a5dc9aaf34,ThromboVision is a biomedical company committe...,"[('thrombovision', 0.1093899921747511), ('medi...","(['medical diagnosis', 'critical cardiovascula..."
fortressware,f1a53b32-a132-1764-9e2d-54e5324e94fc,"Fortressware, Inc. provides solutions to secur...","[('access policies', 0.08053299604810367), ('f...","(['access policies', 'fortress capsule', 'virt..."


In [89]:
### Import the articles

In [44]:
# Get the list of paths to articles files and their sizes
paths = [(company, os.path.join(basedir, "News_1", "".join([company.upper(), ".json"]))) for company in k_companies_list]
paths = [path + (os.stat(path[1]).st_size,) for path in paths]
# Sort by sizes
paths = sorted(paths, key=lambda x: x[2], reverse=True)

full_size = 0
for f_size in paths:
    full_size = full_size + f_size[2]
print("appr. " + str(round(full_size/1024/1024/1024)) + " GB is the full size of our dataset")

appr. 22 GB is the full size of our dataset


In [ ]:
'''
from jsonstreamer import JSONStreamer 
streamer = JSONStreamer()
for index_f, path in enumerate(paths):
    with open(path[1]) as file: 
        for line in file: 
            streamer.consume(line)
'''

In [ ]:
'''
def load_json(filename):
    with open(filename, 'rb') as fd:
        parser = ijson.parse(fd)
        return parser
def objects(self,file):
    key = '-'
    for prefix, event, value in ijson.parse(file):
        if prefix == '' and event == 'map_key':  # found new object at the root
            key = value  # mark the key value
            builder = ObjectBuilder()
        elif prefix.startswith(key):  # while at this key, build the object
            builder.event(event, value)
            if prefix == key+".item" and event == 'end_map':  # found the end of an object at the current key, yield
                yield key, builder.value 
'''

In [209]:
import pprint
# Choose the keyphrase approach
kp_approach = "kp_embedrank"
num_articles = 0
all_articles_dict = dict()
articles = list()
regex1 = re.compile('[%s]' % re.escape(string.punctuation))
regex2 = re.compile('[%s]' % re.escape("\n"))
regex3 = re.compile('[%s]' % re.escape(string.punctuation + "\n"))

if not os.path.exists(os.path.join(basedir, 'temp_data', 'dataset')):
    os.makedirs(os.path.join(basedir, 'temp_data', 'dataset'))

for index_f, path in enumerate(paths):
    #if 500 > index_f > 460:
        
    
    print(str(datetime.datetime.now()) + path[0])   

    counter = 0

    with open(path[1], 'rb') as fd:
        for parsed_object in ijson.items(fd, "item"):

            counter = counter + 1
            if counter > 3:
                break

            # Check if the article is in English
            try:
                if 'ENGLISH' not in set(parsed_object['language']):
                    continue
            except KeyError:
                pass

            # Save the object
            try:   
                article = parsed_object['content'].lower()
                article_dict = dict()
                num_articles = num_articles + 1

                if any(keyphrase in article for keyphrase in set(k_companies_df.loc[path[0], kp_approach])):
                    pass
                else:
                    continue

                paragraphs = article.split("\n\n")
                paragraphs = [regex2.sub(' ', paragraph) for paragraph in set(paragraphs)
                             if len(regex1.sub(' ', paragraph).split()) > 20 and path[0] in paragraph]
                if paragraphs:
                    article_dict["company"] = path[0]
                    article_dict["content"] = paragraphs
                    articles.append(article_dict)
                
            except KeyError:
                continue


    #if articles:
    #    all_articles_dict[path[0]] = articles
    fd.close()
#print(all_articles_dict["udemy"])

2018-07-27 13:28:25.528787brightline
2018-07-27 13:28:25.534091wherefor
2018-07-27 13:28:25.537787musx
2018-07-27 13:28:25.547507crono
2018-07-27 13:28:25.550489gridco
2018-07-27 13:28:25.552966lookback
2018-07-27 13:28:25.557524enervault
2018-07-27 13:28:25.560026energysavvy
2018-07-27 13:28:25.562427humon
2018-07-27 13:28:25.572621muut
2018-07-27 13:28:25.576742bidgely
2018-07-27 13:28:25.581050zqgame
2018-07-27 13:28:25.597659fleetmatics
2018-07-27 13:28:25.600019sungevity
2018-07-27 13:28:25.602510determina
2018-07-27 13:28:25.604496dealertrack
2018-07-27 13:28:25.607324crowdstrike
2018-07-27 13:28:25.609900declara
2018-07-27 13:28:25.612379approva
2018-07-27 13:28:25.615436motopia
2018-07-27 13:28:25.617295unu
2018-07-27 13:28:25.620143stepout
2018-07-27 13:28:25.625746plazes
2018-07-27 13:28:25.627791upsnap
2018-07-27 13:28:25.638573quic
2018-07-27 13:28:25.640846leverton
2018-07-27 13:28:25.643441cassatt
2018-07-27 13:28:25.645761dayak
2018-07-27 13:28:25.704998solfocus
2018-07-

2018-07-27 13:28:26.567112ovascience
2018-07-27 13:28:26.569515smartasset
2018-07-27 13:28:26.571936zoomin
2018-07-27 13:28:26.574219birst
2018-07-27 13:28:26.576288mitro
2018-07-27 13:28:26.580051automattic
2018-07-27 13:28:26.582477mediamath
2018-07-27 13:28:26.585040stryd
2018-07-27 13:28:26.587411kace
2018-07-27 13:28:26.589761datalocker
2018-07-27 13:28:26.592535taboola
2018-07-27 13:28:26.594825synacor
2018-07-27 13:28:26.596975viveve
2018-07-27 13:28:26.599535metacarta
2018-07-27 13:28:26.601738quantcast
2018-07-27 13:28:26.604699freshdesk
2018-07-27 13:28:26.606959webscale
2018-07-27 13:28:26.609256suniva
2018-07-27 13:28:26.611580zetta
2018-07-27 13:28:26.614085savari
2018-07-27 13:28:26.616332qualtrics
2018-07-27 13:28:26.618438budz
2018-07-27 13:28:26.620982xirrus
2018-07-27 13:28:26.622979indemand
2018-07-27 13:28:26.625866kleer
2018-07-27 13:28:26.627836kurtosys
2018-07-27 13:28:26.630180ambarella
2018-07-27 13:28:26.632314klout
2018-07-27 13:28:26.635290getback
2018-07-27

2018-07-27 13:28:27.403874ouya
2018-07-27 13:28:27.407564winster
2018-07-27 13:28:27.410124infolinks
2018-07-27 13:28:27.414766noom
2018-07-27 13:28:27.417156opsource
2018-07-27 13:28:27.419368genee
2018-07-27 13:28:27.421432simfy
2018-07-27 13:28:27.425487zuora
2018-07-27 13:28:27.427909sivi
2018-07-27 13:28:27.430100soasta
2018-07-27 13:28:27.432495ardelyx
2018-07-27 13:28:27.434702springsource
2018-07-27 13:28:27.436804modsy
2018-07-27 13:28:27.439386quickpay
2018-07-27 13:28:27.441723napatech
2018-07-27 13:28:27.443919actifio
2018-07-27 13:28:27.446551cardlytics
2018-07-27 13:28:27.448610gooddata
2018-07-27 13:28:27.451834luxtera
2018-07-27 13:28:27.454279solidfire
2018-07-27 13:28:27.456200meebo
2018-07-27 13:28:27.458917facio
2018-07-27 13:28:27.461260podtech
2018-07-27 13:28:27.467584glympse
2018-07-27 13:28:27.469882invincea
2018-07-27 13:28:27.472526spreadshirt
2018-07-27 13:28:27.475114yub
2018-07-27 13:28:27.477401centrl
2018-07-27 13:28:27.480801portero
2018-07-27 13:28:27.

2018-07-27 13:28:28.216931uship
2018-07-27 13:28:28.220450panjo
2018-07-27 13:28:28.223129fooda
2018-07-27 13:28:28.225912amberpoint
2018-07-27 13:28:28.228000zenoss
2018-07-27 13:28:28.230903spoton
2018-07-27 13:28:28.233710payoneer
2018-07-27 13:28:28.236041desti
2018-07-27 13:28:28.238169nodality
2018-07-27 13:28:28.240549arteris
2018-07-27 13:28:28.242420dailydeal
2018-07-27 13:28:28.244538superfish
2018-07-27 13:28:28.246677houseparty
2018-07-27 13:28:28.249289clarabridge
2018-07-27 13:28:28.251348acumatica
2018-07-27 13:28:28.253425precisionhawk
2018-07-27 13:28:28.255580carecloud
2018-07-27 13:28:28.258074cloudbees
2018-07-27 13:28:28.260503mapmyfitness
2018-07-27 13:28:28.262459invuity
2018-07-27 13:28:28.264859infinio
2018-07-27 13:28:28.267455comentis
2018-07-27 13:28:28.269489novacea
2018-07-27 13:28:28.271385socialtext
2018-07-27 13:28:28.273455internetstores
2018-07-27 13:28:28.275464drobo
2018-07-27 13:28:28.277668bevi
2018-07-27 13:28:28.301392vidyard
2018-07-27 13:28:28

2018-07-27 13:28:29.212907fitocracy
2018-07-27 13:28:29.215320mojio
2018-07-27 13:28:29.217483humacyte
2018-07-27 13:28:29.219844sovi
2018-07-27 13:28:29.222252squareone
2018-07-27 13:28:29.224440parstream
2018-07-27 13:28:29.227179bluestacks
2018-07-27 13:28:29.229912adjudica
2018-07-27 13:28:29.231973vestigo
2018-07-27 13:28:29.234080altheadx
2018-07-27 13:28:29.236011yapta
2018-07-27 13:28:29.237947ilypsa
2018-07-27 13:28:29.239977wispry
2018-07-27 13:28:29.242412conversocial
2018-07-27 13:28:29.244530fundera
2018-07-27 13:28:29.247850authy
2018-07-27 13:28:29.250488nextnav
2018-07-27 13:28:29.252462hellosoft
2018-07-27 13:28:29.256009pluggedin
2018-07-27 13:28:29.267247aternity
2018-07-27 13:28:29.270440mojix
2018-07-27 13:28:29.273415flixster
2018-07-27 13:28:29.275533mendix
2018-07-27 13:28:29.278080mimoon
2018-07-27 13:28:29.280225emotient
2018-07-27 13:28:29.282603eevent
2018-07-27 13:28:29.285002tectura
2018-07-27 13:28:29.287127ebuddy
2018-07-27 13:28:29.289340groupme
2018-07

...

2018-07-27 13:28:48.215593changenexus
2018-07-27 13:28:48.226522sitebrains
2018-07-27 13:28:48.228066dynadec
2018-07-27 13:28:48.229904viralninjas
2018-07-27 13:28:48.231441callvine
2018-07-27 13:28:48.233444beetmobile
2018-07-27 13:28:48.235037kloudnation
2018-07-27 13:28:48.236620mapistry
2018-07-27 13:28:48.238120zursh
2018-07-27 13:28:48.239707hitchedpic
2018-07-27 13:28:48.241336greenboxny
2018-07-27 13:28:48.243047zootrock
2018-07-27 13:28:48.244962careerlark
2018-07-27 13:28:48.246637musistic
2018-07-27 13:28:48.248631hunite
2018-07-27 13:28:48.249885plumzi
2018-07-27 13:28:48.252002futurederm
2018-07-27 13:28:48.253962envirokure
2018-07-27 13:28:48.255785accuitis
2018-07-27 13:28:48.257807evertune
2018-07-27 13:28:48.259069onetwosee
2018-07-27 13:28:48.260770chewse
2018-07-27 13:28:48.262768givenext
2018-07-27 13:28:48.264282chobolabs
2018-07-27 13:28:48.265925veritweet
2018-07-27 13:28:48.267360paykind
2018-07-27 13:28:48.268693adfreeq
2018-07-27 13:28:48.270289ttwick
2018-07-

2018-07-27 13:28:48.621230chirpvision
2018-07-27 13:28:48.622665sharemagnet
2018-07-27 13:28:48.624135amstatz
2018-07-27 13:28:48.625672publicearth
2018-07-27 13:28:48.627188stuffbuff
2018-07-27 13:28:48.628481discoverly
2018-07-27 13:28:48.630422worldhistoryproject
2018-07-27 13:28:48.631955rapidrabbit
2018-07-27 13:28:48.633491wemonitor
2018-07-27 13:28:48.634941netccm
2018-07-27 13:28:48.636306farmivore
2018-07-27 13:28:48.637827stockpilz
2018-07-27 13:28:48.639786cityspade
2018-07-27 13:28:48.647767mywerx
2018-07-27 13:28:48.649657looksima
2018-07-27 13:28:48.651264ignitefeedback
2018-07-27 13:28:48.652703weplann
2018-07-27 13:28:48.653993airfordable
2018-07-27 13:28:48.655736enovalys
2018-07-27 13:28:48.657166videopixie
2018-07-27 13:28:48.658540onovative
2018-07-27 13:28:48.660281schoolinks
2018-07-27 13:28:48.661891lumatix
2018-07-27 13:28:48.663092phorent
2018-07-27 13:28:48.664862shopalytic
2018-07-27 13:28:48.666370ranktab
2018-07-27 13:28:48.668241prizmiq
2018-07-27 13:28:48

2018-07-27 13:28:49.025177gigzolo
2018-07-27 13:28:49.028663gamaby
2018-07-27 13:28:49.031110teneology
2018-07-27 13:28:49.033429pocketsuite
2018-07-27 13:28:49.034651snowvation
2018-07-27 13:28:49.035850pingpad
2018-07-27 13:28:49.036992endoinsight
2018-07-27 13:28:49.038206venuefox
2018-07-27 13:28:49.039536gutenbergz
2018-07-27 13:28:49.040817mogotix
2018-07-27 13:28:49.042040optoatmospherics
2018-07-27 13:28:49.043345retailmls
2018-07-27 13:28:49.044783cartnav
2018-07-27 13:28:49.046172lawkick
2018-07-27 13:28:49.047542juiceqube
2018-07-27 13:28:49.049006binaryvr
2018-07-27 13:28:49.050230iperlane
2018-07-27 13:28:49.051546appslingr
2018-07-27 13:28:49.055011wearpoint
2018-07-27 13:28:49.057330cureeo
2018-07-27 13:28:49.058738windensity
2018-07-27 13:28:49.059967itsplatonic
2018-07-27 13:28:49.061340conjecta
2018-07-27 13:28:49.062550engiver
2018-07-27 13:28:49.063753zangzing
2018-07-27 13:28:49.064961rezora
2018-07-27 13:28:49.066286userengage
2018-07-27 13:28:49.067564huntforce
2

2018-07-27 13:28:49.432601sqft
2018-07-27 13:28:49.433784oper
2018-07-27 13:28:49.434705chai
2018-07-27 13:28:49.435635trac
2018-07-27 13:28:49.436666tra
2018-07-27 13:28:49.437680grandis
2018-07-27 13:28:49.438730uconnect
2018-07-27 13:28:49.439754monthlys
2018-07-27 13:28:49.440811taggs
2018-07-27 13:28:49.441820panna
2018-07-27 13:28:49.442830pirc
2018-07-27 13:28:49.443855zeel
2018-07-27 13:28:49.444883jolie
2018-07-27 13:28:49.445897goshi
2018-07-27 13:28:49.446912arka
2018-07-27 13:28:49.447906nello
2018-07-27 13:28:49.448946pwa
2018-07-27 13:28:49.449923moshi
2018-07-27 13:28:49.450931nerdies
2018-07-27 13:28:49.451946gro
2018-07-27 13:28:49.452985mika
2018-07-27 13:28:49.453998lia
2018-07-27 13:28:49.455014serps
2018-07-27 13:28:49.456027linksy
2018-07-27 13:28:49.457045preferral
2018-07-27 13:28:49.458038continuent
2018-07-27 13:28:49.459050wanamaker
2018-07-27 13:28:49.460059behr
2018-07-27 13:28:49.461041cme
2018-07-27 13:28:49.462053schad
2018-07-27 13:28:49.463067putney
20

2018-07-27 13:28:49.836744pulpix
2018-07-27 13:28:49.837867visibiz
2018-07-27 13:28:49.838848seambliss
2018-07-27 13:28:49.839833sensobi
2018-07-27 13:28:49.840832juicycanvas
2018-07-27 13:28:49.841818secondmic
2018-07-27 13:28:49.842772sendtask
2018-07-27 13:28:49.843718analyticmate
2018-07-27 13:28:49.844722crowdfeed
2018-07-27 13:28:49.845666infikno
2018-07-27 13:28:49.846646commonkey
2018-07-27 13:28:49.847629lookallure
2018-07-27 13:28:49.848631whoapi
2018-07-27 13:28:49.849614playerduel
2018-07-27 13:28:49.850590myrealkarma
2018-07-27 13:28:49.851571ajungo
2018-07-27 13:28:49.852546eventcheq
2018-07-27 13:28:49.853529salesintel
2018-07-27 13:28:49.854507cityciv
2018-07-27 13:28:49.855484mediaroost
2018-07-27 13:28:49.856460tabsprint
2018-07-27 13:28:49.857457omniref
2018-07-27 13:28:49.858431punctil
2018-07-27 13:28:49.859409localbonus
2018-07-27 13:28:49.860380datasquid
2018-07-27 13:28:49.861364prestobox
2018-07-27 13:28:49.862345exchangery
2018-07-27 13:28:49.863321slinkset
20

In [ ]:
# Old code for json library, too slow as it reads every file into RAM
'''
for index_f, file in enumerate(paths):
    #if index_f > 0:
    #    break
    
    print(file)
    print(index_f)

    try:
        articles = pd.read_json(file[1])
    except ValueError:
        print("empty/faulty file")
        continue

    if "content" not in articles.columns:
        print("no content")
        continue

    articles.loc[:, "company"] = file[0]
    articles.loc[:, "uuid"] = k_company_df.loc[file[0].lower(), "uuid"]
    articles.loc[:, "description"] = k_company_df.loc[file[0].lower(), "description"]

    articles.dropna(subset=["content"], inplace=True)

    texts = [regex.sub('', i).lower() for i in articles.loc[:, "content"]]
    keyphrases = ast.literal_eval(k_company_df.loc[file[0].lower(), "kp_topicrank"])
    num_articles = num_articles + len(texts)

    # Check whether the any of the keywords exist in the text
    bi = [1 
             if any([keyphrase in text 
                     for (keyphrase, prob) in (keyphrases)]) and 250 < len(str.split(text)) < 500 
             else 0
            for text in texts]

    articles.loc[:, "kp_topicrank_bi"] = bi
    articles.dropna(subset=["content"], inplace=True)

    all_articles = all_articles.append(articles, ignore_index=True, sort=False)
    
'''  

In [227]:
tmp=[]

for i in articles:
    if i not in tmp:
        tmp.append(i)
num_dup = len(articles)-len(tmp)
print(str(num_dup) + " articles are duplicates")
articles = tmp

435 articles are duplicates


In [228]:
print(str(len(articles)/(num_articles-num_dup)*100) + "% of articles (" +
      str(len(articles)) + " out of " + str(num_articles-num_dup) +
      ") are containing corresponding keyphrases extracted from the descriptions of the companies and contain the name of the company in the relevant paragraphs")

28.322969750806486% of articles (6058 out of 21389) are containing corresponding keyphrases extracted from the descriptions of the companies and contain the name of the company in the relevant paragraphs


In [ ]:
# Save entire dictionary with articles
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open(os.path.join(basedir, 'temp_data', 'articles'), 'wb') as fp:
    pickle.dump(articles, fp)

In [230]:
with open (os.path.join(basedir, 'temp_data', 'articles'), 'rb') as fp:
    articles = pickle.load(fp)

In [216]:
print(json.dumps(articles[0:50], sort_keys=True, indent=4))

[
    {
        "company": "crono",
        "content": [
            "commission europ\u00e9enne/europese commissie, 1049 bruxelles/brussel, belgique/belgi\u00eb - tel. +32 22991111 office: dm24 05/076 - tel. direct line +32 229-+32 229-76970 filip.cornelis@ec.europa.eu european commission directorate-general for mobility and transport directorate e - aviation the director brussels, 15 november 2017 move/e4/pb summary of the conclusions of the air safety committee (asc) meeting established by regulation (ec) 3922/1991 held in brussels on 13-15 november 2017 30th update of the eu air safety list draft commission implementing regulation amending commission regulation (ec) no 474/2006 of 22 march 2006 as regards the list of air carriers which are banned from operating or are subject to operational restrictions within the union. i. hearings: \uf0b7 hearings were held with the following airlines / authorities on the basis of the right of defence established in article 7 of regulation (ec) n

In [225]:
len(tmp)

6058

In [231]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)